In [16]:
import csv
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch import optim
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [17]:
import pandas as pd
import numpy as np
df = pd.read_csv('./dataset/H1_nav.csv')
df.head()
drop = ['FrameNumber', 'Participant', 'Dataset', 'ViewFrame']
df.drop(drop, inplace =True, axis = 1)

In [18]:
df.head()

,x,y,z,HMDRX,HMDRY,HMDRZ
0,0.0500,1.7868,-1.0947,6.9163,350.8206,359.9912
1,0.0498,1.7871,-1.0951,6.9116,351.1272,0.0094
2,0.0498,1.7872,-1.0955,6.8915,351.3081,0.0221
3,0.0498,1.7870,-1.0961,6.9375,351.5385,359.9731
4,0.0497,1.7865,-1.0969,7.1456,351.7421,359.8803


In [19]:
def timePointList(dataList, timePointBracket):
    tpList = []
    for i in range(0,int(np.ma.count(dataList)/timePointBracket)):
        if(np.ma.count(dataList) < i*timePointBracket):
            #This is the last iteration
            delta = (dataList[-1] - dataList[(i*timePointBracket)])/(np.ma.count(dataList) - (i*timePointBracket))
            tpList.append(delta)
        else:
            delta = (dataList[(i*timePointBracket) + timePointBracket - 1] - dataList[(i*timePointBracket)])/timePointBracket
            tpList.append(delta)
    return tpList

def generateMotionGraphs(dataX, dataY, dataZ, dataYaw, dataPitch, dataRoll, timepoints):
    deltaX = timePointList(dataX,timepoints)
    deltaY = timePointList(dataY,timepoints)
    deltaZ = timePointList(dataZ,timepoints)
    deltaYaw = timePointList(dataYaw, timepoints)
    deltaPitch = timePointList(dataPitch, timepoints)
    deltaRoll = timePointList(dataRoll, timepoints)

    plt.figure(1)
    plt.plot(deltaX, label = '△X')
    plt.plot(deltaY, label = '△Y')
    plt.plot(deltaZ, label = '△Z')
    plt.title("Body Motion Speed")
    plt.legend()
    plt.savefig("BodyMotion.jpg", dpi=200)

    plt.figure(2)
    plt.plot(deltaYaw, label = '△Yaw')
    plt.plot(deltaPitch, label = '△Pitch')
    plt.plot(deltaRoll, label = '△Roll')
    plt.title("Head Motion Speed")
    plt.legend()
    plt.savefig("HeadMotion.jpg", dpi=200)



In [20]:
def create_dataset1(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

def generateDataset(dataArr, look_back = 125):
    dataset=dataArr.reshape(-1, 1)
    train_size = int(len(dataset) * 0.9)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

    trainX, trainY = create_dataset1(train, look_back)
    testX, testY = create_dataset1(test, look_back)
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
    trainX = torch.from_numpy(trainX).float()
    trainY = torch.from_numpy(trainY).float()
    testX = torch.from_numpy(testX).float()
    testY = torch.from_numpy(testY).float()
    return trainX, trainY, testX, testY



In [21]:
class LSTM2(nn.Module):
    def __init__(self, hidden_layers=32, numLayers = 1):
        super(LSTM2, self).__init__()
        self.hidden_layers = hidden_layers
        self.numLayers = numLayers
        self.lstm1 = nn.LSTM(125, self.hidden_layers, batch_first=True, num_layers = numLayers)
        self.linear = nn.Linear(self.hidden_layers, 1)
        
    def forward(self, inputVal):
        hidden_state = torch.randn(self.numLayers, inputVal.size(0), 32).float()
        cell_state = torch.randn(self.numLayers, inputVal.size(0), 32).float()
        hidden = (hidden_state, cell_state)
        h_t, c_t = self.lstm1(inputVal.float(), hidden)
        outputs = self.linear(h_t) # output from the last FC layer
        return outputs



In [22]:
def training_loop(n_epochs, model, optimiser, loss_fn, train_input, train_target):
    model.train()
    trainLoss = []
    train_target = train_target.reshape(train_target.size(0), 1, 1)
    for i in range(n_epochs):
        optimiser.zero_grad()
        out = model(train_input)
        loss = loss_fn(out, train_target)
        loss.backward()    
        optimiser.step()
        trainLoss.append(loss)
        print("Step: {}, Loss: {}".format(i, loss))
    return trainLoss

def eval_model(model, loss_fn, test_input, test_target):
        model.eval()
        test_target = test_target.reshape(test_target.size(0), 1, 1)
        with torch.no_grad():
            pred = model(test_input)
            loss = loss_fn(pred, test_target)
            return loss, pred
        



In [23]:
def trainModel(num_epochs, model, optimiser, criterion, trainX, trainY, plotsFolderName, colName):
    model = model.float()
    train_loss = training_loop(num_epochs, model, optimiser, criterion, trainX, trainY)
    lossArr = []
    for i in range(0,len(train_loss)):
        lossArr.append(train_loss[i].item())
    plt.plot(lossArr, label = "Training Loss")
    plt.legend()
    plt.savefig(plotsFolderName+colName+"trainLoss.jpg", dpi=200)
    plt.close()


In [24]:
def evaluateModel(model, criterion, testX, testY, plotsFolderName, colName):
    test_loss, prediction = eval_model(model, criterion, testX, testY)
    print(test_loss.item())
    plt.figure(figsize=(8, 6), dpi=100)
    plt.scatter(testY, prediction.reshape(prediction.size(0)))
    plt.xlabel("Actual Value")
    plt.ylabel("Predicted Value")
    plt.savefig(plotsFolderName+colName+"scatter.jpg", dpi=200)
    plt.close()

In [26]:
for i in range(1,2):
    fileName = r"dataset/H" + str(i) +  ".csv"
    resultsFolder = r"dataset/H" + str(i) + "_nav/"
    plotsFolderName = r"dataset/H" + str(i) + "_nav/plots/"
    df = pd.read_csv (fileName)
    
    for (columnName, columnData) in df.iteritems():
        print("Predicting for " + columnName)
        dataCol = df[columnName].to_numpy()
        trainDataX, trainDataY, testDataX, testDataY = generateDataset(dataCol)
        
        model = LSTM2(numLayers = 10)
        criterion = nn.MSELoss()
        optimiser = optim.Adam(model.parameters(), lr=0.01)
        trainModel(100, model, optimiser, criterion, trainDataX, trainDataY,plotsFolderName,columnName)
        torch.save(model.state_dict(), resultsFolder+columnName+"savedModel.pt")
        evaluateModel(model, criterion, testDataX, testDataY, plotsFolderName, columnName)



Predicting for x
Step: 0, Loss: 0.2251625657081604
Step: 1, Loss: 0.2168758660554886
Step: 2, Loss: 0.21010085940361023
Step: 3, Loss: 0.20663990080356598
Step: 4, Loss: 0.20437437295913696
Step: 5, Loss: 0.203459694981575
Step: 6, Loss: 0.20214691758155823
Step: 7, Loss: 0.20075364410877228
Step: 8, Loss: 0.1997332125902176
Step: 9, Loss: 0.19994626939296722
Step: 10, Loss: 0.1991107016801834
Step: 11, Loss: 0.19890093803405762
Step: 12, Loss: 0.1988217532634735
Step: 13, Loss: 0.19883635640144348
Step: 14, Loss: 0.19873616099357605
Step: 15, Loss: 0.19839806854724884
Step: 16, Loss: 0.19801650941371918
Step: 17, Loss: 0.19789515435695648
Step: 18, Loss: 0.19750601053237915
Step: 19, Loss: 0.19785916805267334
Step: 20, Loss: 0.19775508344173431
Step: 21, Loss: 0.19787341356277466
Step: 22, Loss: 0.19780349731445312
Step: 23, Loss: 0.19755910336971283
Step: 24, Loss: 0.19781582057476044
Step: 25, Loss: 0.19788585603237152
Step: 26, Loss: 0.19814078509807587
Step: 27, Loss: 0.1978202611